In [ ]:
import pandas as pd
import os
import ast
from Bio import SeqIO
import json

In [ ]:
#make json dictionaries for each specie containing prot_id and gene (gene-genename)
for specie in os.listdir('/home/collot/stage/collot/collot/out_stats/databases/'):
    specie_dir = str(os.path.join('/home/collot/stage/collot/collot/out_stats/databases/' + specie))
    db = str(specie[:16])  #db name is db_GCF
    db = gffutils.FeatureDB(specie_dir)
    cds_genes = str(specie[3:16]) #cds dict name is GCF
    cds_genes = {}
    for cds in db.features_of_type('CDS'): 
        for gene_name in cds.attributes["gene"]:
            if cds.id.startswith('cds-'):
                key = cds.id[4:]
            else: 
                key = cds.id 
            cds_genes[key] = gene_name       
    file_path = '/home/collot/stage/collot/collot/out_ortho/dictionaries/'+ specie[3:16]+'.json'
    with open(file_path, 'w') as fp:
        json.dump(cds_genes, fp)


In [ ]:

#group all species dictionaries into one (key to species dictionaries = genome_id) 
dictionaries = {}
for ortho_dict in os.listdir('/home/collot/stage/collot/collot/out_ortho/dictionaries/'):
    dict_path = '/home/collot/stage/collot/collot/out_ortho/dictionaries/' + ortho_dict
    with open(dict_path) as prot_dict:
        dictionaries[ortho_dict[:-5]] = json.load(prot_dict)


In [23]:

#get orthologs dataframe with prot_id   
df_prot_orth = pd.read_csv('/home/collot/stage/collot/collot/out_ortho/prot_oto_homo_sapiens.tsv', sep='\t', index_col=0)
df_prot_orth = df_prot_orth.dropna(ignore_index=True)

df_species = pd.read_csv('/home/collot/stage/collot/collot/out_stats/df_species.tsv',sep='\t')
    

In [ ]:
#df with HS gene and orthologs genes 
df_genes_ortho = pd.DataFrame(columns=['main', 'orthologs'])
for index, row in df_prot_orth.iterrows() : 
    query_prot = df_prot_orth.loc[index,'protein_id']
    for ortho_dict_name, prot in dictionaries.items(): 
            tax_id = str(df_species.loc[df_species['Assembly Accession'].str[:13]==ortho_dict_name, 'Organism Taxonomic ID'].iloc[0])
            query_gene = prot[query_prot] + '_' + tax_id
            df_genes_ortho.loc[index,'main'] = query_gene

    ortho_list = []
    prot_ortho_list=list(str(df_prot_orth.loc[index,'orthologs_protein_id']).split(sep=','))
    for i in range(len(prot_ortho_list)): 
        for ortho_dict_name, prot in dictionaries.items(): 
            if prot_ortho_list[i] in prot : 
                tax_id = str(df_species.loc[df_species['Assembly Accession'].str[:13]==ortho_dict_name, 'Organism Taxonomic ID'].iloc[0])
                gene_ortho = prot[prot_ortho_list[i]] + '_' + tax_id
                ortho_list.append(gene_ortho)
    df_genes_ortho.loc[index, 'orthologs']=ortho_list

pd.DataFrame.to_csv(df_genes_ortho,'/home/collot/stage/collot/out_ortho/genes_orthologs_homo_sapiens.csv' )

In [ ]:
#for one specie 
def write_fasta():
    df_species = pd.read_csv('/home/collot/stage/collot/collot/out_stats/df_species.tsv',sep='\t')
    df_genes_ortho = pd.read_csv('/home/collot/stage/collot/collot/out_ortho/genes_orthologs_homo_sapiens.csv', index_col=0)

    dictionaries_seq={}
    for gen_dir in os.listdir('/home/collot/stage/collot/PrimateData/ncbi_dataset/data/'):
        if gen_dir.startswith('GCF'):
            path_to_dir = str('/home/collot/stage/collot/PrimateData/ncbi_dataset/data/'+ gen_dir + '/')
            path_list=[]
            for file in os.listdir(path_to_dir):
                path_list.append(str('/home/collot/stage/collot/PrimateData/ncbi_dataset/data/' + gen_dir + '/' + file))
            fasta_genome = path_list[0]
            dictionaries_seq[gen_dir] = SeqIO.to_dict(
                SeqIO.parse(fasta_genome, "fasta")
            )

    df_genes_ortho['orthologs'] = df_genes_ortho['orthologs'].apply(ast.literal_eval) #only in a script (no notebook)
    genome_dict = {}
    for index, row in df_genes_ortho.iterrows():
        ortho_fasta = []
        all_genes = [row['main']] + row['orthologs']
        fasta_path = f"/home/collot/stage/collot/collot/out_ortho/fasta_homo_sapiens/{row['main']}.fasta"
        for ortho in all_genes:
            tax = int(ortho.split('_')[1])
            genome = df_species.loc[df_species['Organism Taxonomic ID']==tax,'Assembly Accession'].iloc[0]
            if genome not in genome_dict:
                path = f"/home/collot/stage/collot/collot/out_stats/output_dataframes/{genome}/df_exons"
                genome_dict[genome] = pd.read_csv(path, index_col=0)
            df = genome_dict[genome]
            gene_id = f"gene-{ortho.split('_')[0]}"
            gene_df = df[df['gene'] == gene_id]
            for gene_index, gene_row in gene_df.iterrows():
                exons_list = ast.literal_eval(gene_row['exons'])
                header = f">{gene_row['gene'][5:]}_{str(exons_list[0])[5:]}_{tax}"
                chromosome = gene_row['chromosome']
                borders = [int(gene_row['start']) - 1, int(gene_row['end'])]
                specie_dictionary = dictionaries_seq[genome]
                record = specie_dictionary[chromosome]
                if gene_row['strand'] == '-':
                    exon_seq = record.seq[borders[0]:borders[1]].reverse_complement()
                else:
                    exon_seq = record.seq[borders[0]:borders[1]]
                entry = f"{header}\n{exon_seq}"
                ortho_fasta.append(entry)

        with open(fasta_path, 'w') as f:
            f.write('\n'.join(ortho_fasta))

if __name__=='__main__' : 
    write_fasta()